In [1]:
import ee
ee.Authenticate()  # You'll need to follow the instructions to authenticate
ee.Initialize()

In [22]:
# Define Central Valley polygon using your coordinates
central_valley = ee.Geometry.Polygon([[
  [-119.821696,  36.917661],
  [-119.151455,  36.491729],
  [-118.962469,  35.903247],
  [-118.730887,  35.238055],
  [-118.839257,  35.122308],
  [-118.883280,  34.950407],
  [-119.001362,  35.036397],
  [-119.279275,  35.023953],
  [-118.839257,  35.122308],
  [-120.012630,  35.606365],
  [-119.909518,  35.953074],    
  [-120.724549,  36.737623],
]])

# Filter Sentinel-1 ImageCollection over Central Valley
sentinel1 = ee.ImageCollection('COPERNICUS/S1_GRD') \
              .filterBounds(central_valley) \
              .filter(ee.Filter.eq('instrumentMode', 'IW')) \
              .filter(ee.Filter.eq('orbitProperties_pass', 'DESCENDING')) \
              .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV')) \
              .filterDate('2022-01-01', '2024-03-31')  # Adjust main time window if needed

# Select two images for interferogram with specific time windows
image1 = sentinel1.filterDate('2022-01-01', '2022-01-15').first()  # First time window
image2 = sentinel1.filterDate('2023-02-01', '2024-02-15').first()  # Second time window

# Create interferogram by calculating the phase difference and selecting 'VV' band
def calculate_interferogram(img1, img2):
    return img2.subtract(img1).select('VV')

interferogram = calculate_interferogram(image1, image2)

# Define visualization parameters for the single band (VV)
viz_params = {
    'min': -25,
    'max': 1,
    'palette': ['blue', 'yellow', 'red']
}

# Import geemap for visualization
import geemap
Map = geemap.Map()
Map.centerObject(central_valley, 10)
Map.addLayer(interferogram, viz_params, 'InSAR VV Interferogram')
Map

Map(center=[36.08102049165581, -119.5884170941839], controls=(WidgetControl(options=['position', 'transparent_…

In [23]:
# Export the interferogram to Google Cloud Storage (GCS)
export_task = ee.batch.Export.image.toCloudStorage(
    image=interferogram,
    description='InSAR_Interferogram_Central_Valley',
    bucket='My-bucket-name-Behrooz',  # Replace with your GCS bucket name
    fileNamePrefix='InSAR_Interferogram',  # File prefix for the exported file
    scale=30,  # Set resolution in meters
    region=central_valley,
    maxPixels=1e12,  # Adjust based on your dataset
    fileFormat='GeoTIFF'
)

# Start the export task
export_task.start()

print("Export task started. Check Google Cloud Storage for the result.")

Export task started. Check Google Cloud Storage for the result.
